In [2]:
import argparse
import os
from pprint import pprint


from anago.utils import load_data_and_labels,NERSequence
from anago.models import BiLSTMCRF,load_model,save_model
from anago.preprocessing import IndexTransformer#, ELMoTransformer
from anago.trainer import Trainer
from anago.tagger import Tagger

from lang_util.spell import SpellCorrector


spell = SpellCorrector()

Using TensorFlow backend.


In [9]:
##Input
dir_path = os.getcwd()
train_data = os.path.realpath(os.path.join(dir_path,"../data/train.iob"))
valid_data = os.path.realpath(os.path.join(dir_path,"../data/test.iob"))
weights_file = os.path.realpath(os.path.join(dir_path,"../models/weights.h5"))
params_file = os.path.realpath(os.path.join(dir_path,"../models/params.json"))
preprocessor_file = os.path.realpath(os.path.join(dir_path,"../models/preprocess.bin"))
no_char_feature = 50

dropout = 0.5
no_use_crf = False
char_emb_size = 25
word_emb_size = 150
char_lstm_units = 25
word_lstm_units = 200

epochs = 5

In [10]:
print('Loading dataset...')
x_train, y_train = load_data_and_labels(train_data)
x_valid, y_valid = load_data_and_labels(valid_data)

Y = set()
for yt in y_train:
    for y in  (list(set(yt))):
        Y.add( y )



Loading dataset...


In [11]:

print('Transforming datasets...')
p = IndexTransformer(use_char=no_char_feature)
p.fit(x_train, y_train)

print ("label size:",p.label_size , len(Y))
pprint(p._label_vocab._token2id)

Transforming datasets...
label size: 112 111
{'<pad>': 0,
 'B-ACTIVITY': 7,
 'B-ADULT': 27,
 'B-ADULT B-MAN': 61,
 'B-ADULT B-WOMAN': 48,
 'B-ANTISYMPTOM': 26,
 'B-BABY': 16,
 'B-BBODY': 3,
 'B-BBODY B-DISEASE': 110,
 'B-CHILD': 24,
 'B-CHILD B-SOMEONE': 103,
 'B-DISEASE': 5,
 'B-DISEASE B-BBODY': 76,
 'B-DISEASE B-SYMPTOM': 96,
 'B-DISEASE B-TEST': 89,
 'B-DISEASE B-WOMAN B-ADULT': 85,
 'B-DURATION': 13,
 'B-FOOD': 18,
 'B-LACTATION': 21,
 'B-LACTATION B-ADULT B-WOMAN': 88,
 'B-LACTATION B-WOMAN': 73,
 'B-LACTATION B-WOMAN B-ADULT': 107,
 'B-MAN': 37,
 'B-MAN B-ADULT': 39,
 'B-PREGNANT': 10,
 'B-PREGNANT B-ADULT B-WOMAN': 84,
 'B-PREGNANT B-WOMAN': 58,
 'B-PREGNANT B-WOMAN B-ADULT': 57,
 'B-PRIMARY': 41,
 'B-PROMIL': 32,
 'B-SOMEONE': 17,
 'B-SOMEONE B-ADULT': 53,
 'B-SOMEONE B-ADULT B-MAN': 56,
 'B-SOMEONE B-ADULT B-WOMAN': 63,
 'B-SOMEONE B-BABY': 25,
 'B-SOMEONE B-BBODY': 97,
 'B-SOMEONE B-CHILD': 23,
 'B-SOMEONE B-MAN': 79,
 'B-SOMEONE B-MAN B-ADULT': 34,
 'B-SOMEONE B-SYMPTOM': 8

In [13]:
print('Building a model.')
model = BiLSTMCRF(char_embedding_dim=char_emb_size,
                  word_embedding_dim=word_emb_size,
                  char_lstm_size=char_lstm_units,
                  word_lstm_size=word_lstm_units,
                  char_vocab_size=p.char_vocab_size,
                  word_vocab_size=p.word_vocab_size,
                  num_labels=p.label_size,
                  dropout=dropout,
                  use_char=no_char_feature,
                  use_crf=not no_use_crf)

model, loss = model.build()
model.compile(loss=loss, optimizer='adam')


print('Training the model...')
trainer = Trainer(model, preprocessor=p)
trainer.train(x_train, y_train, x_valid, y_valid,epochs=epochs,verbose=2,initial_epoch=1)




print('Saving the model...')
save_model(model,weights_file, params_file)    
p.save(preprocessor_file)

   

Building a model.
Training the model...
Epoch 1/5


KeyboardInterrupt: 

In [ ]:
text = []
text.append("kalau saat masa subur saya dan sumi melakukan snggama terputus apakah mungkin masih bisa haml")
#text = "raynaldy kelebihan berat badan yang bisa menyebabkan diabetes"
# text.append("masa subur sore dok. saya mau tanya, tanggal 05 juli 2018 nanti saya menikah. hari ini 21 juni 2018 saya haid hari pertama untuk periode juni. kira-kira masa subur saya tanggal berapa? karena saya dan calon sami berencana langsung memiliki momongan..")
# text.append("ambyen dok saya berusia 20thn \
#  namun ambeyen saya sudah sangat besar dan tidak dapat masuk kedalam dubur lagi \
#  saya pernah periksakan namun hanya diberikan obat \
#  tapi tidak ada reaksi \
#  bagaimana ini dok \
#  cara menyembuhkan wasir \
#  tanpa ambil resiko operasi .\
#  dan makanana apa saja yang harus dihindari")

# text.append("bekas jerawat dok ngilangin bekas jerawat pake apa yah? bekas jerawat saya di seblah pipi kanan gak ilang2.\
# trus mau tanya jg kalau pelembab di pake setiap hari buat kulit yg ada bekas jerawat aman gk? wlpn itu gak keluar rumah")

# text.append("thyroid Dok, tiroid bisa menyebabkan jantung berdebar? Apa yg harus saya lakukan jika kambuh?")
# text.append("klnjar tiroid dok kenapa yah smenjak saya mmiliki kelenjar tiroid saya slalu mrasa gelisah suasana hati kadang berubah apakah iitu ada hubunganya dengan benjolan saya")
text.append("dok anak saya umur 2th 6 bulan kemasukan air tlinganya lalu saya baw ke klinik bpjs dan dberi obt puyr dan amoxcilin dan sekarang baik obat maupun amoxcilin sudah habis diminum tp tlinga msh keluar cairan agak kental jadi saya harus bagaimana dok")
text.append("Kulit Bu kalau obat dermatix apakah bisa menghilangkan luka yg belang istilahnya bisa mengembalikan kulit yg brlang menjadi kulit lagi gak bu?")
text.append("wajah dok penyebab kulit kusam terjadi karena apa dok dan cara mengatasinya agar kuliy wajah terlihat cerah")
text.append("penyakit anus keluar cairan bau 4 hr sudah keponakan saya terbaring di tempat tdr mula nya sakit kepala demam terus panas dingin 2 hr ini ko keluar cairan dari anus yg mengeluarkan bau tidak sedap")
text.append("skrang ni saya lagi hamil 4bln 2week,  prtnyaannya knpa yah kulit saya trlihat menghitam.")
text.append("ever whit buat bumil dok aman g jika saya pke pemutih badan ever white")
text.append("ibu mertua saya diabetes 3 bulan")
text.append("paman saya kantong telurnya bernanah")
print('Tagging a sentence...')

for t in text:
    t = spell.validate(t)
    print (t)
    res = tagger.analyze(t)
    pprint(res['entities'])
    #proba = tagger.predict_proba(t)
    #pprint(proba)
    #print(len(proba))
    #print(len(res['words']))